In [5]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key
api_key = os.getenv('GOOGLE_API_KEY')

# Import the necessary function to load GooglePalm
import langchain.llms as llms

# Use the internal function to import GooglePalm
GooglePalm = llms._import_google_palm()

# Initialize the GooglePalm instance with the API key
llm = GooglePalm(google_api_key=api_key, temperature=0.7)

# Print the instance to verify
print(llm)


GooglePalm
Params: {}


In [6]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

/Users/rohansaxena/Desktop/AI Engineer/Week 31-32 - LLM & Langchain/q-a_ai/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
I0000 00:00:1721843093.828763 24233229 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721843093.839297 24233229 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


**Samosas, oh samosas,**
**So golden and crispy, so full of flavor.**
**I love your flaky crust,**
**And your spicy filling.**
**You're the perfect snack,**
**Any time of day.**


In [7]:
essay = llm("write email requesting refund for electronic item")
print(essay)

Dear [Name of Customer Service Representative],

I am writing to request a refund for my [product name] that I purchased on [date of purchase]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product page on your website.

I would like to request a full refund for the product. I would also like to request that you send me a replacement product.

I would appreciate it if you could process my refund as soon as possible. I can be reached at [email address] or [phone number] if you have any questions.

Thank you for your time and consideration.

Sincerely,
[Your name]


In [8]:
from langchain.chains import RetrievalQA

from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# Specify the correct relative path to the CSV file
file_path = './codebasics_faqs.csv'

# Create the loader instance with the correct file path and encoding
loader = CSVLoader(file_path=file_path, source_column="prompt", encoding="latin1")

# Store the loaded data in the 'data' variable
data = loader.load()

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

class DebugHuggingFaceInstructEmbeddings(HuggingFaceInstructEmbeddings):
    def __init__(self, **kwargs):
        print(f"Initializing with kwargs: {kwargs}")
        super().__init__(**kwargs)

instructor_embeddings = DebugHuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

Initializing with kwargs: {'model_name': 'hkunlp/instructor-large'}
load INSTRUCTOR_Transformer
max_seq_length  512


/Users/rohansaxena/Desktop/AI Engineer/Week 31-32 - LLM & Langchain/q-a_ai/myenv/lib/python3.10/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this

In [11]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
 
e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [13]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

/Users/rohansaxena/Desktop/AI Engineer/Week 31-32 - LLM & Langchain/q-a_ai/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
 Document(metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless pr

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [15]:
chain('Do you provide job assistance and also do you provide job gurantee?')

/Users/rohansaxena/Desktop/AI Engineer/Week 31-32 - LLM & Langchain/q-a_ai/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer 

In [16]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "Yes, we provide virtual internship and currently we don't offer EMI payments.",
 'source_documents': [Document(metadata={'source': 'Do you provide any virtual internship?', 'row': 14}, page_content='prompt: Do you provide any virtual internship?\nresponse: Yes'),
  Document(metadata={'source': 'Do we have an EMI option?', 'row': 13}, page_content='prompt: Do we have an EMI option?\nresponse: No'),
  Document(metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(metadata={'source': 'How can I contact the instructors for any doubts/support?', 'row': 5}, page_content='prompt: How can I contact the instructors for any do